In [46]:
# Sports Analytics: Final Semester Project & Portfolio

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
years = list(range(2008, 2025))  # 2008–2024 inclusive
base_url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/"

all_matches = []

for year in years:
    url = base_url + f"atp_matches_{year}.csv"
    print(f"Loading {url} ...")
    df_year = pd.read_csv(url)
    df_year["match_year"] = year
    all_matches.append(df_year)

matches_2008_2024 = pd.concat(all_matches, ignore_index=True)
print("Merged shape (raw):", matches_2008_2024.shape)



Loading https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2008.csv ...
Loading https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2009.csv ...
Loading https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2010.csv ...
Loading https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2011.csv ...
Loading https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2012.csv ...
Loading https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2013.csv ...
Loading https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2014.csv ...
Loading https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2015.csv ...
Loading https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2016.csv ...
Loading https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2017.csv ...
Loading https://raw.

In [49]:
# Filter to completed matches only: non-null score, no WO/RET/etc.
if "comment" in matches_2008_2024.columns:
    mask_completed = matches_2008_2024["comment"].isna()
    matches_2008_2024 = matches_2008_2024[mask_completed]

matches_2008_2024 = matches_2008_2024[matches_2008_2024["score"].notna()]

print("Shape after completed-match filter:", matches_2008_2024.shape)

# Save merged + basic-filtered dataset
output_filename = "atp_matches_2008_2024_merged.csv"
matches_2008_2024.to_csv(output_filename, index=False)
print("Merged dataset saved as:", output_filename)


Shape after completed-match filter: (48779, 50)
Merged dataset saved as: atp_matches_2008_2024_merged.csv


In [50]:
df = pd.read_csv("atp_matches_2008_2024_merged.csv")
print("Reloaded df shape:", df.shape)
df.head()



Reloaded df shape: (48779, 50)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,match_year
0,2008-1536,Madrid Masters,Hard,48,M,20081012,1,105208,NaN,NaN,...,42.0,26.0,17.0,3.0,5.0,54.0,719.0,27.0,1120.0,2008
1,2008-1536,Madrid Masters,Hard,48,M,20081012,2,103888,NaN,NaN,...,14.0,15.0,8.0,1.0,4.0,25.0,1145.0,59.0,694.0,2008
2,2008-1536,Madrid Masters,Hard,48,M,20081012,3,104259,NaN,NaN,...,19.0,8.0,8.0,5.0,9.0,31.0,1065.0,46.0,805.0,2008
3,2008-1536,Madrid Masters,Hard,48,M,20081012,4,103852,NaN,NaN,...,31.0,22.0,10.0,4.0,5.0,40.0,865.0,48.0,777.0,2008
4,2008-1536,Madrid Masters,Hard,48,M,20081012,5,103812,NaN,Q,...,36.0,14.0,13.0,4.0,8.0,73.0,579.0,26.0,1123.0,2008


In [51]:
# 1. Ensure completed matches (defensive re-check)
df = df[df['score'].notna()]
if 'comment' in df.columns:
    df = df[df['comment'].isna()]

# 2. Remove doubles matches (keep singles only)
df = df[df["tourney_level"] != "D"]

# 3. Remove qualifying and round-robin rounds
df = df[~df['round'].isin(['Q1', 'Q2', 'Q3', 'Q', 'RR'])]

# 4. Create unique match key (one row = one ATP singles match)
df['unique_key'] = df['tourney_id'].astype(str) + "_" + df['match_num'].astype(str)

# 5. Standardize surface labels and convert tourney_date to datetime
df['surface'] = df['surface'].str.strip().str.title()
df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d')

print("Shape after singles + rounds + key + standardization:", df.shape)




Shape after singles + rounds + key + standardization: (43748, 51)


In [52]:
print("Row count after filters:", len(df))
print("Duplicate unique keys:", df['unique_key'].duplicated().sum())


Row count after filters: 43748
Duplicate unique keys: 0


In [53]:
# Remove any rows where serve points is zero (just in case)
df = df[(df['w_svpt'] > 0) & (df['l_svpt'] > 0)]

# Winner serve-return balance
df['w_srv_ret_balance'] = (df['w_1stWon'] + df['w_2ndWon']) / df['w_svpt']

# Loser serve-return balance
df['l_srv_ret_balance'] = (df['l_1stWon'] + df['l_2ndWon']) / df['l_svpt']

# Efficiency difference (winner - loser)
df['efficiency_diff'] = df['w_srv_ret_balance'] - df['l_srv_ret_balance']

print("Shape after feature engineering:", df.shape)
df[['w_srv_ret_balance', 'l_srv_ret_balance', 'efficiency_diff']].head()




Shape after feature engineering: (43366, 54)


,w_srv_ret_balance,l_srv_ret_balance,efficiency_diff
0,0.696970,0.715789,-0.018820
1,0.759259,0.568627,0.190632
2,0.723404,0.465517,0.257887
3,0.819672,0.616279,0.203393
4,0.678571,0.574713,0.103859


In [54]:
df[['w_ace', 'l_ace', 'w_df', 'l_df']].describe()



,w_ace,l_ace,w_df,l_df
count,43366.000000,43366.000000,43366.000000,43366.000000
mean,7.161509,5.278628,2.574090,3.246506
std,5.691372,4.998485,2.234089,2.464529
min,0.000000,0.000000,0.000000,0.000000
25%,3.000000,2.000000,1.000000,1.000000
50%,6.000000,4.000000,2.000000,3.000000
75%,10.000000,7.000000,4.000000,4.000000
max,113.000000,103.000000,26.000000,26.000000


In [55]:
df.isna().mean().sort_values(ascending=False)



,0
winner_entry,0.853180
loser_entry,0.762026
loser_seed,0.742194
winner_seed,0.530738
minutes,0.036872
loser_ht,0.003643
loser_rank_points,0.002306
loser_rank,0.002306
winner_ht,0.000530
winner_rank,0.000415


In [56]:
# Validation Gates

In [57]:
df.isna().mean().sort_values(ascending=False)



,0
winner_entry,0.853180
loser_entry,0.762026
loser_seed,0.742194
winner_seed,0.530738
minutes,0.036872
loser_ht,0.003643
loser_rank_points,0.002306
loser_rank,0.002306
winner_ht,0.000530
winner_rank,0.000415


In [58]:
# Clean Dataset!!

In [59]:
print("Row count after cleaning:", len(df))
print("Duplicate unique keys:", df['unique_key'].duplicated().sum())

print("\nMissingness by column (top 15):")
print(df.isna().mean().sort_values(ascending=False).head(15))


Row count after cleaning: 43366
Duplicate unique keys: 0

Missingness by column (top 15):
winner_entry          0.853180
loser_entry           0.762026
loser_seed            0.742194
winner_seed           0.530738
minutes               0.036872
loser_ht              0.003643
loser_rank_points     0.002306
loser_rank            0.002306
winner_ht             0.000530
winner_rank           0.000415
winner_rank_points    0.000415
w_SvGms               0.000023
l_SvGms               0.000023
tourney_date          0.000000
winner_ioc            0.000000
dtype: float64


In [60]:
# Data Dictionary

In [61]:
df.to_csv("atp_matches_2008_2024_clean.csv", index=False)
print("Clean dataset exported as: atp_matches_2008_2024_clean.csv")



Clean dataset exported as: atp_matches_2008_2024_clean.csv
